# Evaluating Responses from Image, Text, and Combined Indexes

***This notebook works best with the `conda_python3` on the `ml.t3.large` instance***.

---

This notebook does as follows:

1. Use the CSV file generated in the previous notebook to run evaluations on how the response quality is, based on the context, and the target responses that are given

1. Record the ROUGE, COSINE, BLEU scores, and for subjective evaluation, use an `LLM as a judge`(in this case, ClaudeV3 Sonnet)

1. Record the results for all kinds of responses from text only index, image only index, and combined (from both the text as well as the image index) from `OpenSearch`

## Step 1. Setup

Install the required Python packages and import the relevant files.

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
# import the libraries that are needed to run this notebook
import os
import re
import ray
import time
import glob
import json
import yaml
import time
import nltk
import boto3
import codecs
import base64
import logging
import requests
import botocore
import sagemaker
import numpy as np
import pandas as pd
import globals as g
from numpy import dot
from pathlib import Path
from nltk.tag import pos_tag
from typing import List, Dict
from numpy.linalg import norm
from litellm import completion ## support for text generation models on bedrock
from rouge_score import rouge_scorer
from nltk.tokenize import word_tokenize
from bedrock_utils import get_bedrock_client
from requests_auth_aws_sigv4 import AWSSigV4
from nltk.translate.bleu_score import sentence_bleu
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from utils import get_cfn_outputs, get_bucket_name, download_image_files_from_s3, get_text_embedding

In [ ]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
if ray.is_initialized():
    ray.shutdown()
ray.init()

In [ ]:
# global constants
CONFIG_FILE_PATH = "config.yaml"
# read the config yaml file
fpath = CONFIG_FILE_PATH
with open(fpath, 'r') as yaml_in:
    config = yaml.safe_load(yaml_in)
logger.info(f"config read from {fpath} -> {json.dumps(config, indent=2)}")

In [ ]:
!pygmentize globals.py

### Get all the responses from the evaluation dataset in a dataframe to calculate ROUGE/COSINE and `LLM as a Judge` evaluation metrics

Here, we fetch responses from the image index, image index to get a question bank created that acts as a curated dataset

In [ ]:
if config['metrics_dir']['dir_name'] is not None:
    metrics_fpath = os.path.join(config['metrics_dir']['dir_name'], config['eval_qna_dataset_info']['updated_eval_file'])
    metric_files = glob.glob(metrics_fpath, recursive=True)
    logger.info(f"there are {len(metric_files)} files in {metrics_fpath}")
    for file in metric_files:
        eval_df = pd.read_csv(file) 
        eval_df = eval_df.loc[:, ~eval_df.columns.str.startswith('Unnamed')]

eval_df.head(10)

### ROUGE & Cosine Similarity Scores for multimodal completions:
---

Here, the amazon.titan-embed-text-v1 is used to get the embeddings of texts. To use a different embeddings model, change the model in the embeddings_model_info and modify this function.

In [ ]:
from typing import Optional
MAX_TEXT_LEN_FOR_EMBEDDING: int = config['embeddings_model_info']['max_text_len_for_embedding']
bedrock: Optional[get_bedrock_client] = None

def get_embedding(text: str, modelId: str=config['embeddings_model_info'].get('model'), accept: str='application/json', contentType: str='application/json'):
    """
    Generates embeddings for the responses from the image/text indexes and the target responses if any are
    provided in the dataset
    """
    global bedrock
    if bedrock is None:
        bedrock = get_bedrock_client()
    body = json.dumps({"inputText": text[:MAX_TEXT_LEN_FOR_EMBEDDING]})
    response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body.get('embedding')
    token_count = response_body.get('inputTextTokenCount')
    return embedding, token_count

def get_cosine_similarity(text1: str, text2: str) -> float:
    """
    This function calculates the cosine similarity between the image/text indexes and the target responses if any
    """
    A,_ = get_embedding(text1)
    B,_ = get_embedding(text2)
    cosine = dot(A, B)/(norm(A)*norm(B))
    return cosine

def get_rouge_l_score(completion: str, golden: str) -> float:
    """
    This function calculates the rouge-l score between the image/text indexes and the target responses (if any)
    """
    rouge_metric_selection: str = config['embeddings_model_info']['rouge_metric_selection']
    scorer = rouge_scorer.RougeScorer([rouge_metric_selection])
    scores = scorer.score(golden, completion)
    return round(scores[rouge_metric_selection].fmeasure, 4)

In [ ]:
target_response_key: str = config['eval_qna_dataset_info']['target_response_key']
def compare_completions(row, index_type):
    """
    Generates the rouge and cosine similarity scores for chapter titles and original titles
    """
    if (row.get(index_type) and row.get(target_response_key) is not None) and (pd.notna(row.get(index_type)) and pd.notna(row.get(target_response_key))):
        logger.info(f"{index_type} response: {row[index_type]}, Target response: {row[target_response_key]}")
        rouge_l_score = get_rouge_l_score(row[index_type], row[target_response_key])
        cosine_sim = get_cosine_similarity(row[index_type].lower(), row[target_response_key].lower())
        return pd.Series([rouge_l_score, cosine_sim])
    else:
        logger.info(f'ROUGE, Cosine similarity and Bleu scores cannot be computed since original responses are not provided in the dataset')
        rouge_l_score, cosine_sim = None, None

if target_response_key in eval_df.columns:
    for metric in config['embeddings_model_info']['get_quantitative_metrics_on']:
        eval_df[[f'{metric}_rouge_l_f1_score', f'{metric}_cosine_similarity']] = eval_df.apply(lambda row: compare_completions(row, index_type=metric), axis=1)
else:
    logger.info('No evaluation metrics available since target responses are not provided in the dataset.')

In [ ]:
if config['eval_qna_dataset_info']['target_response_key'] in eval_df.columns:
    eval_df.rename(columns = {'Response':'target_response', 'text_response': 'text_only_response', 'img_response': 'img_only_response'}, inplace = True)
    eval_df = eval_df.drop(columns=['image_and_text_source', 'text_source', 'img_source'])
# Construct the file path
metrics_dir: str = config['metrics_dir']['dir_name']
rouge_cosine_file_path = os.path.join(metrics_dir, config['metrics_dir']['eval_score_dataset'])

eval_df.to_csv(rouge_cosine_file_path, index=False)
eval_df.head(10)

### Use `LLM as a Judge` to evaluate responses from different indexes - WIP

### Title Evaluation: Using LLM as a Judge in the loop
---

In this portion:

Responses generated by each index and combined are evaluated on relevance and best match by `Claude Sonnet/Your model of choice`. Prompt for the model that acts as a judge in the loop can be viewed in: `eval_template.txt`. Edit and review this prompt based on the use case and criteria for subjective evaluation.

The role of the model acting as a judge it to compare the responses generated by each index to a target response. It provides information on the `best selected response`, `response source` (from the text only index/image only index/or both indexes), and an `explanation` of its selection, with an in depth analysis of comparison between other responses and why it chose the one it did. 

A final evaluation metric is calculated that shows the distribution of the selected responses and their respective response sources. This will give a judgement call of which multimodal strategy to use in production ready workloads.

Note: For more information on the use of having a Model act as a judge, view: https://huggingface.co/learn/cookbook/en/llm_judge

#### Prepare the evaluation prompt payloads

Here, the evaluation prompt template is used by the LLM judge to evaluate different chapter titles and suggest the most suitable title based on the evaluation criteria mentioned in the prompt template.

In [ ]:
def prepare_eval_prompts(row):
    """
    This function evaluates the prompts by incorporating all of the responses generated by various indexes into the evaluation prompt template.
    """
    # represents the eval template used by the model judge
    eval_template: Optional[str] = None
    processed_eval_template: Optional[str] = None
    candidate_responses: List[str] = []
    try:
        # file path to the eval template
        eval_template_path: str = os.path.join(config['pdf_dir_info']['prompts'], config['eval_qna_dataset_info'].get('prompt_template'))
        with open(eval_template_path, "r") as f:
            eval_template = f.read()
            logger.info(f"evaluation prompt template recorded: {eval_template}")
    except FileNotFoundError:
        print(f"Error: Evaluation template not found at {eval_template_path}")
    logger.info(f"question: {row['Query']}")
    logger.info(f"original_response: {row['target_response']}")
    for column in row.index:
        if column.endswith("_response") and column != "target_response":
            response_source = column.split("_response")[0]
            candidate_response = row[column]
            candidate_responses.append(f"\n<{response_source}>\n{candidate_response}\n</{response_source}>\n")
    processed_eval_template = eval_template.format(
        question=row['Query'], 
        original_response=row['target_response'],
        candidate_responses="\n".join(candidate_responses)
    )
    return processed_eval_template

#### Add evaluation prompt as a column into a df with respective response index sources and responses to send into the Model for further evaluation in the loop

In [ ]:
if eval_df is not None:
    eval_df['eval_prompt'] = eval_df.apply(lambda r: prepare_eval_prompts(r), axis=1)
    logger.info("preparing the evaluation prompt templates for the LLM judge....")
else:
    logger.error(f"Model evaluation dataset is not available to process.")
llm_as_a_judge_eval_df_f_path = os.path.join(metrics_dir, config['pdf_dir_info']['processed_prompts_for_eval'])
eval_df.insert(0, 'query_id', eval_df.index)
eval_df.to_csv(llm_as_a_judge_eval_df_f_path, index=False)
eval_df.head(10)

### Using `LLM (Claude) as a judge` in the loop to evaluate and narrow down the titles generated by different models of choice

In [ ]:
def llm_judge_json_evaluations(model_id: str, prompt: str):
    # represents the service name
    service_name: str = "bedrock"
    # represents creating the bedrock model to invoke the litellm api for response for titan, llama and claude
    bedrock_model: str = f"{service_name}/{model_id}"
    # represents the current aws region
    aws_region = boto3.Session().region_name 
    # initialize the response dict
    ret = dict(exception = None,
               prompt = prompt,
               completion = None,
               question = None,
               target_response = None, 
               # initializing to 0 since none type throws an error later, this is used to calculate price per token input/output on ODT pricing
               completion_token_count = 0,
               # initializing to 0 since none type throws an error later
               prompt_token_count=0,
               input_token_price = None, 
               output_token_pricing = None,
               model_id = model_id)
    body = ret['prompt']
    os.environ["AWS_REGION_NAME"] = aws_region
    parameters = config['inference_parameters_for_explanations']
    temperature = parameters.get('temperature', 0.1)
    caching = parameters.get('caching', False)
    max_tokens = parameters.get("max_tokens", 500)
    try:
        # Represents calling the litellm completion/messaging api utilizing the completion/embeddings API
        logger.info(f"Invoking {bedrock_model}......")
        response = completion(model=bedrock_model,
                              messages=[{ "content": body,"role": "user"}],
                              temperature=temperature,
                              max_tokens=max_tokens,
                              caching=caching)

        # iterate through the entire model response
        for idx, choice in enumerate(response.choices):
            # extract the message and the message's content from litellm
            if choice.message and choice.message.content:
                # extract the response from the dict
                ret["completion"] = choice.message.content.strip()
        # Extract number of input and completion prompt tokens (this is the same structure for embeddings and text generation models on Amazon Bedrock)
        ret['prompt_token_count'] = response.usage.prompt_tokens
        ret['completion_token_count'] = response.usage.completion_tokens
    except Exception as e:
        logger.error(f"Exception occurred during invoking {model_id}, exception={e}")
        ret['exception'] = e

    logger.info(f"completion: {ret['completion']}")
    return ret

In [ ]:
def get_inference(i: int, row: Dict, total: int, model_info: Dict) -> Dict:
    # save all the responses from the model in a dictionary
    resp: Dict = {}
    print(f"row={row}")
    logger.info(f"row {i}/{total}, prompt_template={model_info['prompt_template']}, model_id={model_info['model']}")
    model_id = model_info['model']
    # create the payload for model inference
    prompt = row['eval_prompt']
    # generate the chapter title based on the given chapter in the prompt 
    resp = llm_judge_json_evaluations(model_id, prompt)
    resp['question'] = row['Query']
    resp['target_response'] = row['target_response']
    # calculate the input and output token price for all of the calls
    resp['input_token_price'] = (resp['prompt_token_count']/1000) * model_info['input_tokens_pricing']
    resp['output_token_pricing'] = (resp['completion_token_count']/1000) * model_info['output_tokens_pricing']
    dir_path = os.path.join(config['pdf_dir_info']['judge_model_eval_completions'], str(row['query_id']), model_id.replace(":", "-"))
    os.makedirs(dir_path, exist_ok=True)
    fpath = os.path.join(dir_path, f"model_evaluation_{row['query_id']}.json")
    logger.info(f"writing response={resp} to {fpath}")
    Path(fpath).write_text(json.dumps(resp, default=str, indent=2))
    logger.info(f"response {i}: {resp}")
    return resp

@ray.remote
def async_get_inference(i: int, row: Dict, total: int, model_info: Dict) -> Dict:
    logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
    logger = logging.getLogger(__name__)
    return get_inference(i, row, total, model_info)

In [ ]:
eval_df = json.loads(eval_df.to_json(orient='records'))
n: int = config['parallel_inference_count']
from typing import List
resp_list: List = []
model_list = config['eval_model_info']
st = time.perf_counter()
logger.info(f"------ running inference for {model_list.get('model')} -----")
list_of_lists = [eval_df[i * n:(i + 1) * n] for i in range((len(eval_df) + n - 1) // n )]
logger.info(f"split input list of size {len(eval_df)} into {len(list_of_lists)} lists")
for idx, l in enumerate(list_of_lists):
    logger.info(f"getting inference for list {idx+1}/{len(list_of_lists)}, size of list={len(l)} ")
    resp_list.extend(ray.get([async_get_inference.remote(i+1, e, len(l), model_list) for i, e in enumerate(l)]))
elapsed_time = time.perf_counter() - st
logger.info(f"------ model={model_list.get('model')} completed in {elapsed_time} ------ ")

In [ ]:
## Represents extracted all metric files
fpath_evaluated_files = os.path.join(config['pdf_dir_info']['judge_model_eval_completions'], "**", "*", "*.json")
eval_metric_files = glob.glob(fpath_evaluated_files, recursive=True)
logger.info(f"there are {len(eval_metric_files)} evaluated files by {config['eval_model_info']['model']} LLM judge in {fpath_evaluated_files}")

In [ ]:
model_evaluation_responses = []
for f in eval_metric_files:
    with open(f, 'r') as file:
        model_evaluation_responses.append(json.loads(file.read()))
# results_df will contain the evaluation responses, including the completion and the model id
results_df = pd.DataFrame(model_evaluation_responses)
results_df = results_df.drop(columns=['exception', 'prompt'])
results_df.head(10)

In [ ]:
def clean_model_eval_json(data):
    """
    This function is to take in json data, and clean it, assign the selected title as outputted by the model evaluator
    """
    try:
        json_data = json.loads(data.replace('\\', '\\\\'))
        return pd.Series({
            'best_match_response': json_data['best_match_response'],
            'response_source': json_data['response_source'],
            'explanation': json_data['explanation'],
        })
    except json.JSONDecodeError:
        return pd.Series({
            'best_match_response': None,
            'response_source': None,
            'explanation': None,
        })

In [ ]:
def tidy_split(df, column, sep=',', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.
    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [ ]:
new_results_df = results_df['completion'].apply(clean_model_eval_json)
new_results_df.insert(0, 'question', new_results_df.index)
new_results_df.insert(1, 'target_response', new_results_df.index)
new_results_df['question'] = results_df['question']
new_results_df['target_response'] = results_df['target_response']
new_exploded_df = tidy_split(new_results_df, 'response_source', sep=',')
new_exploded_df['response_source'] = new_results_df['response_source'].str.replace('<', '').str.replace('>', '')
logger.info(f"All evaluation data is read into a dataframe of shape {results_df.shape}")
processed_prompts_for_eval_path = os.path.join(metrics_dir, config['pdf_dir_info']['llm_as_a_judge_completions'])
new_results_df.to_csv(processed_prompts_for_eval_path, index=False)
# display the selected title, model explanation and the respective golden title in a side by side view
new_results_df.head(10)

In [ ]:
# Compute the percentage of each model selection and reset the index
new_exploded_df['response_source'] = new_exploded_df['response_source'].map(lambda x: x.strip())
response_index_percentage_df = new_exploded_df['response_source'].value_counts(normalize=True).reset_index()
response_distribution_fpath = os.path.join(metrics_dir, config['pdf_dir_info']['index_response_distribution'])
response_index_percentage_df.rename(columns = {'response_source':'index_pick_rate'}, inplace = True)
response_index_percentage_df['index_pick_rate'] *= 100
response_index_percentage_df.to_csv(response_distribution_fpath, index=False)
response_index_percentage_df.head(10)

## Clean Up
